<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [3]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    # tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

In [9]:
def DE(pop_size, iterations, function, limits):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    # function calls = pop_size + iterations * pop_size
    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    # print(X)
    amount_of_layer_1 = int(round(X[0]))
    amount_of_layer_2 = int(round(X[1]))
    amount_of_layer_3 = int(round(X[2]))
    amount_of_one_dence = int(round(X[3]))
    amount_of_one_filters = int(round(X[4]))
    # print(np.int64(np.round(X)))
    # print(amount_of_layer_1, amount_of_layer_2, amount_of_layer_3, amount_of_one_dence, amount_of_one_filters)
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    # model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[1, 4], [0, 4], [0, 4], [1, 4], [16, 128]]


In [ ]:
best_f, best_d = DE(4, 4, func, func_limits)
best_f, best_d

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - Top2: 0.4079 - accuracy: 0.2251 - loss: 2.3337 - val_Top2: 0.2178 - val_accuracy: 0.1159 - val_loss: 3.2192
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - Top2: 0.6096 - accuracy: 0.3807 - loss: 1.6972 - val_Top2: 0.6211 - val_accuracy: 0.3797 - val_loss: 1.8499
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - Top2: 0.6644 - accuracy: 0.4422 - loss: 1.5552 - val_Top2: 0.5193 - val_accuracy: 0.2915 - val_loss: 2.2746
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - Top2: 0.6963 - accuracy: 0.4778 - loss: 1.4544 - val_Top2: 0.6636 - val_accuracy: 0.4366 - val_loss: 1.6269
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - Top2: 0.7112 - accuracy: 0.5005 - loss: 1.3977 - val_Top2: 0.6111 - val_accuracy: 0.4348 - val_loss: 1.7057
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - Top2: 0.7332 - accuracy: 0.5235 - loss: 1.3360 - val_Top2: 0.6754 - val_accuracy: 0.4678 - val_loss: 1.5197
Epoch 7/10

Processing:   0%|                                        | 0/4 [00:00<?, ?step/s]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - Top2: 0.4170 - accuracy: 0.2294 - loss: 2.1945 - val_Top2: 0.2185 - val_accuracy: 0.1183 - val_loss: 4.4656
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - Top2: 0.6103 - accuracy: 0.3854 - loss: 1.6863 - val_Top2: 0.6106 - val_accuracy: 0.3930 - val_loss: 1.7140
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - Top2: 0.6709 - accuracy: 0.4443 - loss: 1.5412 - val_Top2: 0.5889 - val_accuracy: 0.3739 - val_loss: 1.7077
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - Top2: 0.7067 - accuracy: 0.4901 - loss: 1.4284 - val_Top2: 0.5488 - val_accuracy: 0.3328 - val_loss: 2.1009
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - Top2: 0.7276 - accuracy: 0.5135 - loss: 1.3608 - val_Top2: 0.7083 - val_accuracy: 0.5036 - val_loss: 1.3642
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - Top2: 0.7376 - accuracy: 0.5345 - loss: 1.3209 - val_Top2: 0.6412 - val_accuracy: 0.4510 - val_loss: 1.6022
Epoch 7/10

Processing:  25%|██████████                              | 1/4 [50:33<2:31:39, 3033.25s/step]

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 46s 113ms/step - Top2: 0.3110 - accuracy: 0.1614 - loss: 3.2834 - val_Top2: 0.2495 - val_accuracy: 0.1232 - val_loss: 2.4207
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 108ms/step - Top2: 0.4821 - accuracy: 0.2597 - loss: 1.9563 - val_Top2: 0.5499 - val_accuracy: 0.3063 - val_loss: 1.8518
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 108ms/step - Top2: 0.5445 - accuracy: 0.3007 - loss: 1.8417 - val_Top2: 0.5115 - val_accuracy: 0.2823 - val_loss: 2.2281
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 105ms/step - Top2: 0.5826 - accuracy: 0.3452 - loss: 1.7644 - val_Top2: 0.5724 - val_accuracy: 0.3486 - val_loss: 1.7955
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 108ms/step - Top2: 0.6194 - accuracy: 0.3839 - loss: 1.6743 - val_Top2: 0.6446 - val_accuracy: 0.3971 - val_loss: 1.6788
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 108ms/step - Top2: 0.6598 - accuracy: 0.4246 - loss: 1.5848 - val_Top2: 0.5777 - val_accuracy: 0.3676 - val_loss: 1.9447
Epoc

In [ ]:
best_f, best_d

In [ ]:
def func(X):
    amount_of_layer_1 =
    amount_of_layer_2 =
    amount_of_layer_3 =
    amount_of_one_dence =
    amount_of_one_filters = int(round(X[0]))
    Dropout = X[1]
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(Dropout))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[16, 128], [0.01, 0.99]]

In [ ]:
dots = 8
x1_lim = func_limits[0]
x1 = np.linspace(x1_lim[0], x1_lim[1], dots)
x2_lim = func_limits[1]
x2 = np.linspace(x2_lim[0], x2_lim[1], dots)
space = np.meshgrid(x1, x2)
fitness = np.array([[func(np.array([i, j]))] for j in x2] for i in x1)

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)